In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

In [2]:
spark = SparkSession.builder \
    .appName('AV-parking-area-analysis') \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

26/02/10 14:20:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 건축물 대장 데이터

In [3]:
building_path = "../data/building/건축물대장_기본개요_2512.txt"

building_schema = StructType([
    StructField("관리_건축물대장_PK", StringType(), True),
    StructField("관리_상위_건축물대장_PK", StringType(), True),
    StructField("대장_구분_코드", StringType(), True),
    StructField("대장_구분_코드_명", StringType(), True),
    StructField("대장_종류_코드", StringType(), True),
    StructField("대장_종류_코드_명", StringType(), True),
    StructField("대지_위치", StringType(), True),
    StructField("도로명_대지_위치", StringType(), True),
    StructField("건물_명", StringType(), True),
    StructField("시군구_코드", StringType(), True),
    StructField("법정동_코드", StringType(), True),
    StructField("대지_구분_코드", StringType(), True),
    StructField("번", StringType(), True),
    StructField("지", StringType(), True),
    StructField("특수지_명", StringType(), True),
    StructField("블록", StringType(), True),
    StructField("로트", StringType(), True),
    StructField("외필지_수", IntegerType(), True),
    StructField("새주소_도로_코드", StringType(), True),
    StructField("새주소_법정동_코드", StringType(), True),
    StructField("새주소_지상지하_코드", StringType(), True),
    StructField("새주소_본_번", IntegerType(), True),
    StructField("새주소_부_번", IntegerType(), True),
    StructField("지역_코드", StringType(), True),
    StructField("지구_코드", StringType(), True),
    StructField("구역_코드", StringType(), True),
    StructField("지역_코드_명", StringType(), True),
    StructField("지구_코드_명", StringType(), True),
    StructField("구역_코드_명", StringType(), True),
    StructField("생성_일자", StringType(), True)
])

building_df = spark.read \
    .option("header", False) \
    .option("sep", "|") \
    .option("encoding", "UTF-8") \
    .schema(building_schema) \
    .csv(building_path)

In [4]:
# 건축물대장(building_df)의 대지구분코드를 토지정보 PNU 표준(1, 2)으로 변환
building_df = building_df.withColumn("pnu_land_gb", 
    F.when(F.col("대지_구분_코드") == "0", "1")  # 대지(0) -> PNU용(1)
     .when(F.col("대지_구분_코드") == "1", "2")  # 산(1) -> PNU용(2)
     .otherwise(F.col("대지_구분_코드"))         
)

building_df = building_df.withColumn(
    "고유번호",
    F.concat(
        F.col("시군구_코드"),          # 5
        F.col("법정동_코드"),          # 5
        F.col("pnu_land_gb"),       
        F.lpad(F.col("번"), 4, "0"),     # 본번
        F.lpad(F.col("지"), 4, "0")      # 부번
    )
)

In [5]:
building_df = building_df.select("고유번호").distinct()

building_df.count()

5950583

### 토지 소유정보 데이터

In [6]:
ground_own_path = "../data/ground/소유정보/경기도_토지소유정보.csv" 

ground_own_df = spark.read \
    .option("header", "true") \
    .option("encoding", "UTF-8") \
    .csv(ground_own_path)

In [27]:
ground_own_df.printSchema()

root
 |-- 고유번호: string (nullable = true)
 |-- 법정동코드: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 대장구분코드: string (nullable = true)
 |-- 대장구분명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 집합건물일련번호: string (nullable = true)
 |-- 건물동명: string (nullable = true)
 |-- 건물층명: string (nullable = true)
 |-- 건물호명: string (nullable = true)
 |-- 건물실명: string (nullable = true)
 |-- 공유인일련번호: string (nullable = true)
 |-- 기준연월: string (nullable = true)
 |-- 지목코드: string (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 토지면적: string (nullable = true)
 |-- 공시지가: string (nullable = true)
 |-- 소유구분코드: string (nullable = true)
 |-- 소유구분: string (nullable = true)
 |-- 국가기관구분코드: string (nullable = true)
 |-- 국가기관구분: string (nullable = true)
 |-- 소유권변동원인코드: string (nullable = true)
 |-- 소유권변동원인: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 공유인수: string (nullable = true)
 |-- 데이터기준일자: string (nullable = true)
 |-- 원천시도시군구코드: string (nullab

In [7]:
ground_own_df = ground_own_df.select(
    "고유번호", 
    "법정동명", 
    "지번", 
    "소유구분코드", 
    "소유구분",
    "소유권변동원인코드",
    "소유권변동원인",
    "소유권변동일자",
    "공유인수",
    "토지면적",
    "지목코드",
    "지목",
    "공시지가",
    "기준연월"
)

In [8]:
ground_own_df.count()

15571343

In [22]:
ground_own_df.groupBy("고유번호") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .show(10)

[Stage 26:===================================================>    (24 + 2) / 26]

+-------------------+-----+
|           고유번호|count|
+-------------------+-----+
|4111513700103100000|12653|
|4113510700103300000| 7292|
|4136011200140020001| 7015|
|4136011200131980011| 7015|
|4113510700103310000| 6076|
|4111113600108810000| 6067|
|4127110300116390007| 5514|
|4113110100169510000| 5493|
|4117110100113940000| 5356|
|4117110100113930000| 5356|
+-------------------+-----+
only showing top 10 rows


In [23]:
# 1. 윈도우 정의: 고유번호별로 그룹을 짓고, 변동일자를 내림차순(최신순)으로 정렬
window_spec = Window.partitionBy("고유번호").orderBy(F.col("소유권변동일자").desc())

# 2. 최신 순위(rank) 부여 및 필터링
# rank가 1인 데이터만 가져오면 각 필지별 가장 최근의 소유권 변동 상태만 남습니다.
latest_own_df = ground_own_df.withColumn("rank", F.rank().over(window_spec)) \
    .filter(F.col("rank") == 1) \
    .drop("rank")

# 3. 이제 현재 시점의 "필지당 주인 수" 계산
owner_count_per_pnu = latest_own_df.groupBy("고유번호") \
    .count() \
    .orderBy(F.col("count").desc())

# 4. 결과 확인
owner_count_per_pnu.show(10)

[Stage 31:====================================>                    (7 + 4) / 11]

+-------------------+-----+
|           고유번호|count|
+-------------------+-----+
|4137011300105860000| 1836|
|4159026221122010000| 1787|
|4159013500107480000| 1775|
|4122012800118940003| 1612|
|4128111200103670001| 1568|
|4128111200103670000| 1568|
|4145010900111400000| 1559|
|4122011400107330000| 1350|
|4163011400110960000| 1346|
|4163011400108860000| 1226|
+-------------------+-----+
only showing top 10 rows


In [9]:
# 1. 소유 정보에서 필지당 1명만 추출 (최신 소유자)
# row_number는 공동 순위를 허용하지 않아 무조건 한 행만 반환합니다.
window_spec = Window.partitionBy("고유번호").orderBy(F.col("소유권변동일자").desc())

filterd_ground_own_df = ground_own_df \
    .withColumn("row_num", F.row_number().over(window_spec)) \
    .filter(F.col("row_num") == 1) \
    .drop("row_num")

filterd_ground_own_df.count()

5200257

### 토지 특성정보 데이터

In [10]:
ground_char_path = "../data/ground/특성정보/경기도_토지특성정보.csv" 

ground_char_df = spark.read \
    .option("header", "true") \
    .option("encoding", "UTF-8") \
    .csv(ground_char_path)

In [11]:
ground_char_df = ground_char_df.select(
    "고유번호", 
    "용도지역코드1",
    "용도지역명1",
    "용도지역코드2",
    "용도지역명2",
    "토지이용상황코드",
    "토지이용상황",
    "지형높이코드",
    "지형높이",
    "지형형상코드",
    "지형형상",
    "도로접면코드",
    "도로접면"
)

In [12]:
ground_char_df.count()

4951368

In [18]:
ground_char_df.groupBy("고유번호") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .show(5)

[Stage 17:===================================================>     (9 + 1) / 10]

+-------------------+-----+
|           고유번호|count|
+-------------------+-----+
|4111112900101010000|    1|
|4111112900100430003|    1|
|4111112900101130001|    1|
|4111112900100520017|    1|
|4111112900100110000|    1|
+-------------------+-----+
only showing top 5 rows


### 토지 정보 join 및 가공

In [13]:
ground_df = ground_char_df.join(
    filterd_ground_own_df,
    on="고유번호",
    how="inner"
)

ground_df.count()

4906589

In [34]:
ground_df.printSchema()

root
 |-- 고유번호: string (nullable = true)
 |-- 용도지역코드1: string (nullable = true)
 |-- 용도지역명1: string (nullable = true)
 |-- 용도지역코드2: string (nullable = true)
 |-- 용도지역명2: string (nullable = true)
 |-- 토지이용상황코드: string (nullable = true)
 |-- 토지이용상황: string (nullable = true)
 |-- 지형높이코드: string (nullable = true)
 |-- 지형높이: string (nullable = true)
 |-- 지형형상코드: string (nullable = true)
 |-- 지형형상: string (nullable = true)
 |-- 도로접면코드: string (nullable = true)
 |-- 도로접면: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유구분코드: string (nullable = true)
 |-- 소유구분: string (nullable = true)
 |-- 소유권변동원인코드: string (nullable = true)
 |-- 소유권변동원인: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 공유인수: string (nullable = true)
 |-- 토지면적: string (nullable = true)
 |-- 지목코드: string (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 공시지가: string (nullable = true)
 |-- 기준연월: string (nullable = true)



In [14]:
target_jimok = ["08", "09", "11", "13", "28"] 

filtered_ground_df = ground_df.filter(
    (F.col("도로접면") != "맹지") & 
    (F.col("지목코드").isin(target_jimok)) &
    (F.col("토지면적").cast("float") >= 300)
    
)

filtered_ground_df.count()

581377

### 건축물 데이터와 join

In [15]:
final_df = filtered_ground_df.join(
    building_df,
    on="고유번호",
    how="left_anti"
)

final_df.count()

172857

In [19]:
# 1. "용도지역명1" 컬럼으로 그룹화 및 카운트
# 2. 개수가 많은 순서(내림차순)로 정렬
land_use_distribution = final_df.groupBy("용도지역명1") \
    .count() \
    .orderBy(F.col("count").desc())

# 3. 결과 출력 (컬럼 내용이 길 수 있으므로 truncate=False 권장)
land_use_distribution.show(truncate=False)

[Stage 71:=========================================>               (8 + 3) / 11]

+-----------------+-----+
|용도지역명1      |count|
+-----------------+-----+
|계획관리지역     |76162|
|자연녹지지역     |29967|
|보전관리지역     |13864|
|제1종일반주거지역|11395|
|생산관리지역     |9614 |
|제2종일반주거지역|5887 |
|개발제한구역     |5593 |
|일반공업지역     |3700 |
|농림지역         |3508 |
|준주거지역       |2846 |
|일반상업지역     |2711 |
|생산녹지지역     |2166 |
|제1종전용주거지역|1197 |
|준공업지역       |1080 |
|보전녹지지역     |886  |
|제3종일반주거지역|771  |
|자연환경보전지역 |557  |
|근린상업지역     |271  |
|중심상업지역     |250  |
|용도미지정       |166  |
+-----------------+-----+
only showing top 20 rows


In [20]:
final_df.show(10, vertical=True)

26/02/10 14:08:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 83:>                                                         (0 + 1) / 1]

-RECORD 0-----------------------------------------
 고유번호           | 4111112900102940001         
 용도지역코드1      | 13                          
 용도지역명1        | 제1종일반주거지역           
 용도지역코드2      | 00                          
 용도지역명2        | 지정되지않음                
 토지이용상황코드   | 320                         
 토지이용상황       | 주상나지                    
 지형높이코드       | 02                          
 지형높이           | 평지                        
 지형형상코드       | 05                          
 지형형상           | 부정형                      
 도로접면코드       | 05                          
 도로접면           | 중로각지                    
 법정동명           | 경기도 수원시 장안구 파장동 
 지번               | 294-1                       
 소유구분코드       | 01                          
 소유구분           | 개인                        
 소유권변동원인코드 | 03                          
 소유권변동원인     | 소유권이전                  
 소유권변동일자     | 2012-02-15                  
 공유인수           | 0                           
 토지면적           | 350.00             

In [16]:
output_path = "../data/output/final_land_data"

final_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .option("encoding", "UTF-8") \
    .csv(output_path)

26/02/10 14:23:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                